In [195]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import threading
import queue
import random

In [29]:
%%capture
pip install --upgrade firebase-admin 


In [161]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("../jsonFile/service-account.json")
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

In [162]:
# docs = db.collection(u'cguscholar').document('faE3_ksAAAAJ').collection(u'updateTime').order_by(u'h_index.All', direction=firestore.Query.DESCENDING).limit(3).get()
# for doc in docs:
#     print(f'{doc.id} => {doc.to_dict()}')

In [163]:
docs = db.collection(u'cguscholar').order_by(u'updateTime').get()

for doc in docs:
    print(f'{doc.id}')

Rfc8k-EAAAAJ
CRo660MAAAAJ
Ca8f9TEAAAAJ
NAzAaY0AAAAJ
QLxmlf0AAAAJ
6Z9AaHAAAAAJ
lLgQE20AAAAJ
0ZE58lcAAAAJ
ylK19VUAAAAJ
IB3_fzoAAAAJ
1ZO7pHkAAAAJ
bjEpXBoAAAAJ
faE3_ksAAAAJ
-8xAh5IAAAAJ
g-_ZXGsAAAAJ
OrFUuH0AAAAJ
x_wsduUAAAAJ


In [164]:
def currentTime():
    now = datetime.datetime.now()
    currentTime = now.strftime("%Y-%m-%d %H:%M:%S")
    return currentTime

In [189]:
#personal detail
def personalData(soup):
    info = {}
    content = ''.join(soup.find('meta', property = 'og:image')['content']) #list to string

    d = soup.find('div', id = 'gsc_prf_i')

    #name
    info['name'] = d.find('div', id = 'gsc_prf_in').text

    #university
    try:
        info['university'] = d.find('a', class_ = 'gsc_prf_ila').text
    except:
        info['university'] = ' '

    #picture
    info['picture'] = soup.find('div', id = 'gsc_prf_pua').find('img')['src'] 
    
    label = []
    for p in soup.find_all('a', class_='gsc_prf_inta gs_ibl'):

        label.append(p.text)
    info['label'] = label
    info['updateTime'] = currentTime()
    return info


In [166]:
def citeBy(soup):
    citeBy = {}
    citations = {}
    h_index = {}
    i10_index = {}
    def cited(status, value):

        if status / 2 < 1:        
            if status % 2 == 0:
                citations['All'] = value
            else:
                citations['Since2016'] = value
            citeBy['citations'] = citations

        if status / 2 < 2:
            if status % 2 == 0:
                h_index['All'] = value
            else:
                h_index['Since2016'] = value
            citeBy['h_index'] = h_index

        if status / 2 < 3:
            if status % 2 == 0:
                i10_index['All'] = value
            else:
                i10_index['Since2016'] = value
            citeBy['i10_index'] = i10_index
            
    count_d = 0
    for d in soup.find_all('td', class_='gsc_rsb_std'):

        cited(count_d, d.text)
        count_d = count_d + 1       
    
    return citeBy

In [167]:
def result(soup, ID):
    infos = {}
    infos['id'] = ID
    infos['personalData'] = personalData(soup)
    infos['cited'] = citeBy(soup)

    return infos

In [168]:
def personalPage(id):
    url = 'https://scholar.google.com.tw/citations?hl=zh-TW&user=' + id
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return result(soup, id)

In [169]:
import json
def jsontransfer(items):
    data = json.dumps(items)
    jsonStr = json.loads(data) 
    return jsonStr

In [170]:
def updatePersonal(userID):
    items = jsontransfer(personalPage(userID))
    print(items)
    ref = db.collection(u'cguscholar').document((items['id']))
    ref.collection(u'updateTime').document((items['personalData']['updateTime'])).set(items['cited'])
    ref.set(items['personalData'])

In [171]:
def dataCheck(ID):
    users_ref = db.collection(u'cguscholar').document(ID)
    doc = users_ref.get()
    if doc.exists:
        checkTemp = doc.to_dict()
        Timestamp = checkTemp['updateTime']
        return Timestamp
    else:
        return ('Not found')

In [172]:
def expiresCheck(last_update, expires):
    if last_update == 'Not found':
        compare = True
    else:
        expires_date = datetime.datetime.strptime(last_update, "%Y-%m-%d %H:%M:%S")
        compare_date = expires_date + datetime.timedelta(days = expires)
        current_date = datetime.datetime.now()

        compare = compare_date < current_date
    print(compare)
    return compare # compare result
    #過期或Not found為true

In [173]:
expiresCheck(dataCheck('Rfc8k-EAAAAJ'), 1)

False


False

In [193]:
def userDataList(label):
    userDataList = []
    # 建立佇列
    ID_queue = queue.Queue()

    # 將資料放入佇列
    label_ref = db.collection(u'Label-Domain').document(label)
    docs = label_ref.get()
    IDtemp = docs.to_dict()
    for i in range(10):
        expire_time = dataCheck(IDtemp['userID'][i])
        if(expiresCheck(expire_time, 1) or expire_time == 'Not found'):
            ID_queue.put(IDtemp['userID'][i])

    # Worker 類別，負責處理資料
    class Worker(threading.Thread):
      def __init__(self, queue, num):
        threading.Thread.__init__(self)
        self.queue = queue
        self.num = num

      def run(self):
        while self.queue.qsize() > 0:
          # 取得新的資料
          userID = self.queue.get()

          # 處理資料
          #print("Worker %d: %s" % (self.num, userID))
          updatePersonal(userID)

          time.sleep(1)

    # 建立兩個 Worker
    my_worker1 = Worker(ID_queue, 1)
    my_worker2 = Worker(ID_queue, 2)

    # 讓 Worker 開始處理資料
    my_worker1.start()
    my_worker2.start()

    # 等待所有 Worker 結束
    my_worker1.join()
    my_worker2.join()

    print("Done.")
    


In [194]:
label = 'causal_inference'
userDataList(label)

True
True
True
True
True
True
True
True
True
True
{'id': 'dE64QHAAAAAJ', 'personalData': {'name': 'Jonathan Sterne', 'university': 'University of Bristol', 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=dE64QHAAAAAJ&citpid=1', 'label': ['Medical Statistics', 'HIV', 'Evidence Synthesis', 'Epidemiology', 'Causal Inference'], 'updateTime': '2021-09-27 21:43:25'}, 'cited': {'citations': {'All': '126243', 'Since2016': '78013'}, 'h_index': {'All': '131', 'Since2016': '98'}, 'i10_index': {'All': '374', 'Since2016': '306'}}}
{'id': 'DZ-fHPgAAAAJ', 'personalData': {'name': 'Bernhard Schölkopf', 'university': ' ', 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=DZ-fHPgAAAAJ&citpid=2', 'label': ['Machine Learning', 'Causal Inference', 'Artificial Intelligence', 'Computational Photography', 'Statistics'], 'updateTime': '2021-09-27 21:43:25'}, 'cited': {'citations': {'All': '177274', 'Since2016': '80166'}, 'h_index': {'All': '

Exception in thread Thread-474:
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\USER\anaconda3\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "C:\Users\USER\anaconda3\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\USER\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "C:\Users\USER\anaconda3\lib\site-packages\urllib3\connectionpool.py", line

Done.


In [88]:
# jsonFile = open("jsonFile/personalData.json", "w")
# jsonFile.write(jsonStr)
# jsonFile.close()